In [48]:
# enable automatic reloading of imported modules.
%reload_ext autoreload
%autoreload 2
from src.master import MasterProblem
from src.slave import SlaveProblem
from src.slave_he import GeneticAlgorithm
from src.ISlave import *
from pulp import *
import numpy as np
import csv
import time

In [3]:
# Param Definition

# Lettura delle istanze

def readFile(max_length,num_items):
    max_axis_length = 0
    axis_lengths = []
    axis_demands = []

    max_length = max_length
    num_items =num_items
    file_name = f"data/input_data_{max_length}_{num_items}.csv"

    with open(file_name, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        max_axis_length, *data = [list(map(int, row)) for row in reader]

    max_axis_length = max_axis_length[0]
    axis_lengths = [row[0] for row in data]
    axis_demands = [row[1] for row in data]
    num_axes = len(axis_lengths)

    print("Max Axis Length: %s" % max_axis_length)
    # print("Axis Lengths: %s" % axis_lengths)
    # print("Axis Demands: %s\n" % axis_demands)
    print("Axis types: %s\n" % num_axes)
    return (max_axis_length,axis_lengths,axis_demands,num_axes,num_items)

In [55]:

def runColumnGeneration(max_axis_length, axis_lengths, axis_demands, slaveType=SlaveProblem, debugMode=False):
    # Generazione dei file .lp
    SlaveProblem.it = 0
    MasterProblem.it = 0
    
    start_time = time.time()

    # Initial solution (greedy):
    # - Use one axes for each demand of that axes length
    # - Start with a poor solution and expect the total required axes to decrease
    patterns = []
    for i in range(num_axes):
        pattern = np.zeros(num_axes)
        pattern[i] = 1 
        patterns.append(pattern)

    masterProblem = MasterProblem(max_axis_length, axis_lengths, axis_demands, patterns, slaveType)

    relaxed = True
    iteration = 0
    max_iterations = 1000
    consecutive_unchanged_duals = 15
    old_duals = []

    while relaxed and iteration < max_iterations:
        duals = masterProblem.solve()
        status = masterProblem.getStatus()
        if(status =='Infeasible'):
            print("The problem is Infeasible, stopping now")
            return
        
        if old_duals == duals:
            consecutive_unchanged_duals -= 1
        else:
            consecutive_unchanged_duals = 15
        if consecutive_unchanged_duals > 0:
            newPattern = masterProblem.startSlave(duals)
            if(debugMode):
                    print("New duals", duals)
                    print("New pattern added", newPattern)
            if newPattern:
                masterProblem.addPattern(newPattern)
               
        else:
            print(consecutive_unchanged_duals)
            relaxed = False

          

        old_duals = duals
        iteration += 1
    # Solve the master
    masterProblem.setRelaxed(False)
    masterProblem.solve()
    
    end_time = time.time()
    seconds = end_time - start_time


    optimal = masterProblem.getComputedOptimal()
   
    if(debugMode):
        for pattern in masterProblem.getUsedPatterns():
            print("%s used %s times" % (pattern[1], pattern[0]))
        return  masterProblem.getUsedPatterns()
    return optimal,iteration,seconds


In [22]:
# Creazione del problema

def runPLI(axis_lengths,max_axis_length,axis_demands,debug=False):
    prob = LpProblem("1D-Stock-Cut", LpMinimize)

    # Variabili di decisione
    pattern_vars = LpVariable.dicts("Pattern", range(len(axis_lengths)), 0, None, LpContinuous)

    # Funzione obiettivo
    prob += lpSum(pattern_vars)

    # Vincoli
    for i in range(len(axis_lengths)):
        prob += axis_lengths[i] * pattern_vars[i] >= max_axis_length

    for i in range(len(axis_lengths)):
        prob += pattern_vars[i] >= axis_demands[i]

    # Risoluzione del problema
    prob.solve()

    # Stampa della soluzione
    print("Status:", LpStatus[prob.status])
    print("Optimal number of axes to cut:", value(prob.objective))

    # Stampa dei pattern utilizzati
    used_patterns = []
    for i in range(len(axis_lengths)):
        if value(pattern_vars[i]) > 0:
            used_patterns.append((value(pattern_vars[i]), i))
    if(debug):
        print("Used patterns:")
        for pattern in used_patterns:
            print("Pattern", pattern[1], "used", pattern[0], "times")

    prob.writeLP('models/pli/prob.lp')


In [57]:
import pandas as pd

data = [(25,3),(25,12),(50,5),(50,45),(100,46),(250,150),(500,250)]

results = []


for d in data:
    SlaveProblem.it = 0
    MasterProblem.it = 0
    (max_axis_length,axis_lengths,axis_demands,num_axes,num_items) =readFile(d[0],d[1])
    optimal, iteration, seconds = runColumnGeneration(max_axis_length, axis_lengths, axis_demands, SlaveProblem)
    optimal2, iteration2, seconds2 = runColumnGeneration(max_axis_length, axis_lengths, axis_demands, GeneticAlgorithm)

    #runPLI(axis_lengths,max_axis_length,axis_demands)
    # Append the results to the list
    results.append({
        'Algorithm': 'Slave Problem',
        'Max Axis Length': max_axis_length,
        'Axis Lengths': axis_lengths,
        'Axis Demands': axis_demands,
        'Optimal': optimal,
        'Iteration': iteration,
        'Seconds': seconds
    })
    
    results.append({
        'Algorithm': 'Genetic Algorithm',
        'Max Axis Length': max_axis_length,
        'Axis Lengths': axis_lengths,
        'Axis Demands': axis_demands,
        'Optimal': optimal2,
        'Iteration': iteration2,
        'Seconds': seconds2
    })
df = pd.DataFrame(results)
df = df[['Algorithm', 'Max Axis Length', 'Axis Lengths', 'Axis Demands', 'Optimal', 'Iteration', 'Seconds']]
print(df)



Max Axis Length: 25
Axis types: 3



c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


0
0
Max Axis Length: 25
Axis types: 12

0
0
Max Axis Length: 50
Axis types: 5

0
0
Max Axis Length: 50
Axis types: 45

0
0
Max Axis Length: 100
Axis types: 46

0
0
Max Axis Length: 250
Axis types: 150

0
0
Max Axis Length: 500
Axis types: 250

0
            Algorithm  Max Axis Length  \
0       Slave Problem               25   
1   Genetic Algorithm               25   
2       Slave Problem               25   
3   Genetic Algorithm               25   
4       Slave Problem               50   
5   Genetic Algorithm               50   
6       Slave Problem               50   
7   Genetic Algorithm               50   
8       Slave Problem              100   
9   Genetic Algorithm              100   
10      Slave Problem              250   
11  Genetic Algorithm              250   
12      Slave Problem              500   
13  Genetic Algorithm              500   

                                         Axis Lengths  \
0                                        [17, 12, 16]   
1        

In [2]:
import pandas as pd
df = pd.read_csv('output.csv')

df['Axis Lengths'] = df['Axis Lengths'].apply(lambda x: len(x))
df.to_csv('output.csv', index=False)


In [6]:

# Convert seconds to time format
df['Time'] = pd.to_datetime(df['Seconds'], unit='s').dt.strftime('%H:%M:%S.%f')




In [7]:
df['Time']

0     00:00:00.000504
1     00:00:00.000266
2     00:00:00.000991
3     00:00:00.000646
4     00:00:00.000710
5     00:00:00.000405
6     00:00:00.005165
7     00:00:00.005530
8     00:00:00.005413
9     00:00:00.004477
10    00:00:00.019252
11    00:00:00.016290
12    00:00:00.065849
13    00:00:00.046950
Name: Time, dtype: object